In [ ]:
import pandas as pd

import os
data_path = './Kaggle-Datasets/glassesnoglasses/faces-spring-2020'
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [9]:
train_df = pd.read_csv('./Kaggle-Datasets/glassesnoglasses/train.csv')
dir = './Kaggle-Datasets/glassesnoglasses/faces-spring-2020/face-'
train_df['path'] = train_df['id'].apply(lambda x: dir + str(x) + '.png').tolist()
train_df = train_df[['path', 'glasses']]

display(train_df)

,path,glasses
0,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,0
1,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,1
2,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,1
3,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,0
4,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,0
...,...,...
4495,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,0
4496,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,0
4497,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,1
4498,./Kaggle-Datasets/glassesnoglasses/faces-sprin...,1


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore

train_data = ImageDataGenerator(
    horizontal_flip = True,
    rescale = 1.0/255,
    shear_range=0.2,
    zoom_range=0.2)


In [11]:
train_gen = train_data.flow_from_dataframe(
    dataframe=train_df.astype({'glasses': str}),  
    x_col='path',
    y_col='glasses',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
  
)

Found 4500 validated image filenames belonging to 2 classes.


In [12]:
cnn = tf.keras.models.Sequential(name = 'GorNG') 

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape = [150,150,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape = [74,74,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', input_shape = [37,37,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(512, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))

cnn.summary()

/home/mlicious/NeuralNetworks/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1735828369.568369   14152 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "GorNG"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','Precision','Recall'])

In [30]:
cnn.fit(train_gen,epochs = 38)

/home/mlicious/NeuralNetworks/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.6443 - loss: 0.6922
Epoch 2/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.8639 - loss: 0.3406
Epoch 3/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.8633 - loss: 0.3384
Epoch 4/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 999ms/step - accuracy: 0.8864 - loss: 0.2943
Epoch 5/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8813 - loss: 0.3039
Epoch 6/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8880 - loss: 0.2946
Epoch 7/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 997ms/step - accuracy: 0.8838 - loss: 0.2841
Epoch 8/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.8900 - loss: 0.2696
Epoch 9/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 996ms/step - accuracy: 0.8859 - loss: 0.2820
Epoch 10/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 996ms/step - accuracy: 0.8833 - loss: 0.2850
Epoch 11/38
141/141 ━━━━━━━━━━━━━━━━━━━━ 141s 997ms/step - accuracy: 0.8857 - loss: 0.2731
Epoch 12/38
141/141 ━━━━━━